In [108]:
import requests
from bs4 import BeautifulSoup
import string
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [82]:
url = 'https://en.wikipedia.org/wiki/Mass_shootings_in_the_United_States'

response=requests.get(url)
page=response.text

soup=BeautifulSoup(page,"html5lib")

In [83]:
# grab tge table with data
table = soup.find("table")
df = pd.read_html(str(table))
df = df[0]
df.columns = df.iloc[0]
df = df.iloc[1:]

In [84]:
# clean up column names
df.columns = [str(c).strip() for c in df.columns]
df.columns

Index(['nan', 'Incident', 'Year', 'Deaths', 'Injuries',
       'Type of firearm(s) used', 'Ref(s)'],
      dtype='object')

In [85]:
# since some of the data is shifted, identify what rows got shifted by one column
df['shifted'] = [any(c.isalpha() for c in col) for col in df['nan']]

In [86]:
# split into correct and shifted dfs
df_1 = df[df['shifted']==False]
df_2 = df[df['shifted']==True]

In [87]:
df_1 = df_1.iloc[:,1:6].copy()
df_2 = df_2.iloc[:,:5].copy()

In [88]:
df_2.columns = df_1.columns

In [89]:
df = pd.concat([df_1,df_2])

In [90]:
df['Year'] = [int(y) for y in df['Year']]
df['Injuries'] = [int(i.split(' ')[0]) for i in df['Injuries']]
df['Deaths'] = [int(d.split(' ')[0]) for d in df['Deaths']]

In [91]:
df.sort_values(by='Year',ascending=True,inplace=True)
df

,Incident,Year,Deaths,Injuries,Type of firearm(s) used
15,Camden shootings,1949,13,3,Semi-automatic pistol
8,University of Texas tower shooting,1966,18,31,"Semi-automatic rifles, revolver, pistols and s..."
20,Easter Sunday massacre,1975,11,0,Semi-automatic pistols and revolver
16,Wilkes-Barre shootings,1982,13,1,Semi-automatic rifle
17,Wah Mee massacre,1983,13,1,Multiple types of firearms
7,San Ysidro McDonald's massacre,1984,23,19,Multiple types of firearms
22,Palm Sunday massacre,1984,10,0,Semi-automatic pistols
11,Edmond post office shooting,1986,15,6,Semi-automatic pistols
23,GMAC shootings,1990,10,6,Semi-automatic rifle
6,Luby's shooting,1991,24,27,Semi-automatic pistols


In [98]:
# assign a count for each shooting in the same year
df['shooting_count'] = df.groupby(['Year']).cumcount()+1

In [111]:
df

,Incident,Year,Deaths,Injuries,Type of firearm(s) used,shooting_count
15,Camden shootings,1949,13,3,Semi-automatic pistol,1
8,University of Texas tower shooting,1966,18,31,"Semi-automatic rifles, revolver, pistols and s...",1
20,Easter Sunday massacre,1975,11,0,Semi-automatic pistols and revolver,1
16,Wilkes-Barre shootings,1982,13,1,Semi-automatic rifle,1
17,Wah Mee massacre,1983,13,1,Multiple types of firearms,1
7,San Ysidro McDonald's massacre,1984,23,19,Multiple types of firearms,1
22,Palm Sunday massacre,1984,10,0,Semi-automatic pistols,2
11,Edmond post office shooting,1986,15,6,Semi-automatic pistols,1
23,GMAC shootings,1990,10,6,Semi-automatic rifle,1
6,Luby's shooting,1991,24,27,Semi-automatic pistols,1


In [106]:
df['shooting_count'].unique()

array([1, 2, 3])

In [109]:
years = np.arange(1985,2019)

In [112]:
df_to_join = []
for y in years:
    for i in df['shooting_count'].unique():
        df_to_join.append([y,i])

In [115]:
df_to_join = pd.DataFrame(df_to_join, columns=['Year','shooting_count'])

In [131]:
df_to_plot = df_to_join.merge(df, how='left', on=['Year','shooting_count'])

In [133]:
df_to_plot.to_csv('mass_shootings_1985_2018.csv')